<a href="https://colab.research.google.com/github/QiewBie/imdb_reviews_check/blob/main/imdb_reviews_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Спочатку потрібно встановити пакет tensorflowjs через pip.
!pip install tensorflowjs
!pip install TensorFlow==2.15.0
!pip install tensorflow-decision-forests==1.8.1
# Головна проблема - неузгодженість версій бібліотек: у такий спосіб вона розв'язується

  Using cached tensorflowjs-4.22.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
  Using cached tf_keras-2.18.0-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflowjs-4.22.0-py3-none-any.whl (89 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tf-keras
    Found existing installation: tf_keras 2.17.0
    Uninstalling tf_keras-2.17.0:
      Successfully uninstalled tf_keras-2.17.0
  Using cached tensorflow-2.15.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Using cached ml_dtypes-0.2.0-c

In [ ]:
from google.colab import drive

# Монтуємо Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np

tf.keras.backend.clear_session()

# Завантаження та підготовка даних
max_features = 10000  # Розмір словника
max_len = 200        # Максимальна довжина послідовності

# Завантаження набору даних IMDB
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    num_words=max_features
)

# Перетворення послідовностей в однакову довжину
x_train = tf.keras.preprocessing.sequence.pad_sequences(
    x_train, maxlen=max_len
)
x_test = tf.keras.preprocessing.sequence.pad_sequences(
    x_test, maxlen=max_len
)

# Створення моделі
model = tf.keras.Sequential([
    # Шар вбудовування перетворює цілі числа (індекси слів) у вектори фіксованої розмірності
    tf.keras.layers.Embedding(max_features, 128),

    # Двонаправлений LSTM шар для обробки послідовності
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),

    # Другий LSTM шар
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),

    # Щільний шар з dropout для регуляризації
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    # Вихідний шар для бінарної класифікації
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Компіляція моделі
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


17464789/17464789 [==============================] - 0s 0us/step


In [ ]:
# Навчання моделі - рекомендований Runtime T4 GPU
history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)


Epoch 1/10
625/625 [==============================] - 265s 401ms/step - loss: 0.4259 - accuracy: 0.7966 - val_loss: 0.3521 - val_accuracy: 0.8518
Epoch 2/10
625/625 [==============================] - 248s 397ms/step - loss: 0.2369 - accuracy: 0.9099 - val_loss: 0.3079 - val_accuracy: 0.8694
Epoch 3/10
625/625 [==============================] - 247s 396ms/step - loss: 0.1566 - accuracy: 0.9436 - val_loss: 0.3803 - val_accuracy: 0.8630
Epoch 4/10
625/625 [==============================] - 246s 394ms/step - loss: 0.0994 - accuracy: 0.9665 - val_loss: 0.5350 - val_accuracy: 0.8454
Epoch 5/10
625/625 [==============================] - 245s 392ms/step - loss: 0.0793 - accuracy: 0.9739 - val_loss: 0.4689 - val_accuracy: 0.8542


In [ ]:
# Оцінка моделі на тестових даних
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"\nТестова точність: {test_accuracy:.4f}")

782/782 [==============================] - 76s 98ms/step - loss: 0.3176 - accuracy: 0.8684

Тестова точність: 0.8684


In [ ]:
# Функція для передбачення настрою відгуку
def predict_sentiment(text, word_index):
    # Конвертація тексту в послідовність індексів
    encoded_text = tf.keras.preprocessing.text.text_to_word_sequence(text)
    encoded_text = [word_index.get(word, 0) for word in encoded_text]
    # Доповнення послідовності
    encoded_text = tf.keras.preprocessing.sequence.pad_sequences(
        [encoded_text], maxlen=max_len
    )
    # Отримання передбачення
    prediction = model.predict(encoded_text)
    return "Позитивний" if prediction[0] > 0.5 else "Негативний"

In [ ]:
import tensorflow as tf
import json
import numpy as np
import os

# Збереження моделі у форматі .keras (рекомендований спосіб)
try:
    # Створюємо директорію для збереження, якщо вона не існує
    os.makedirs('saved_model', exist_ok=True)

    # Зберігаємо модель у форматі .keras
    model.save('saved_model/imdb_model.keras')
    print("Модель успішно збережена у форматі .keras")


except Exception as e:
    print(f"Помилка при збереженні моделі: {str(e)}")

# Зберігаємо конфігурацію моделі
config = {
    'max_len': max_len,
    'max_features': max_features
}

try:
    # Отримуємо та зберігаємо словник
    word_index = tf.keras.datasets.imdb.get_word_index()
    config['word_index'] = {
        word: idx for word, idx in word_index.items()
        if idx < max_features
    }

    # Зберігаємо конфігурацію у JSON
    with open('saved_model/model_config.json', 'w', encoding='utf-8') as f:
        json.dump(config, f, ensure_ascii=False, indent=2)
    print("Конфігурація успішно збережена")

except Exception as e:
    print(f"Помилка при збереженні конфігурації: {str(e)}")


Модель успішно збережена у форматі .keras
1641221/1641221 [==============================] - 0s 0us/step
Конфігурація успішно збережена


In [ ]:
import tensorflow as tf
import numpy as np
import json
from functools import lru_cache

class SentimentPredictor:
    def __init__(self, model_path='saved_model/imdb_model.keras', config_path='saved_model/model_config.json'):
        # Завантаження моделі та конфігурації при ініціалізації
        # Використовуємо tf.keras.models.load_model для завантаження збереженої моделі
        self.model = tf.keras.models.load_model(model_path)

        # Завантаження конфігурації з JSON файлу
        with open(config_path, 'r', encoding='utf-8') as f:
            self.config = json.load(f)

        # Створюємо оптимізовану функцію передбачення
        # Використовуємо experimental_relax_shapes=True для кращої обробки різних розмірів вхідних даних
        self.predict_fn = tf.function(
            self.model,
            reduce_retracing=True,
            experimental_relax_shapes=True
        )

        # Кешуємо результати токенізації для покращення продуктивності
        self.tokenize = lru_cache(maxsize=1000)(self._tokenize)

    def _tokenize(self, text):
        """Токенізація тексту з використанням словника слів."""
        # Розбиваємо текст на слова та конвертуємо у індекси
        words = tf.keras.preprocessing.text.text_to_word_sequence(text)
        return [self.config['word_index'].get(word, 0) for word in words]

    def _preprocess(self, text):
        """Попередня обробка тексту для подачі в модель."""
        # Токенізуємо текст
        encoded = self.tokenize(text)

        # Доповнюємо послідовність до потрібної довжини
        padded = tf.keras.preprocessing.sequence.pad_sequences(
            [encoded],
            maxlen=self.config['max_len'],
            padding='post'
        )

        # Конвертуємо в тензор з правильним типом даних
        return tf.convert_to_tensor(padded, dtype=tf.int32)

    @tf.function
    def _process_prediction(self, prediction_tensor):
        """Обробка результатів передбачення у вигляді тензора."""
        # Отримуємо скалярне значення з тензора
        # tf.squeeze видаляє зайві розмірності
        scalar_pred = tf.squeeze(prediction_tensor)

        # Визначаємо настрій на основі порогового значення
        is_positive = tf.greater(scalar_pred, 0.5)

        # Обчислюємо впевненість
        confidence = tf.where(
            is_positive,
            scalar_pred,
            1.0 - scalar_pred
        )

        return is_positive, confidence, scalar_pred

    def predict(self, text):
        """Виконання передбачення з правильною обробкою тензорів."""
        try:
            # Підготовка вхідних даних
            input_tensor = self._preprocess(text)

            # Отримання передбачення
            prediction_tensor = self.predict_fn(input_tensor, training=False)

            # Обробка результатів передбачення
            is_positive, confidence, raw_pred = self._process_prediction(prediction_tensor)

            # Конвертуємо тензори в звичайні Python-значення
            return {
                'status': 'success',
                'sentiment': "Позитивний" if is_positive.numpy() else "Негативний",
                'confidence': float(confidence.numpy()),
                'raw_prediction': float(raw_pred.numpy())
            }

        except Exception as e:
            return {
                'status': 'error',
                'error': str(e)
            }

def analyze_texts(texts, predictor=None):
    """Аналіз списку текстів з створенням предиктора за необхідності."""
    # Створюємо предиктор, якщо він не був переданий
    if predictor is None:
        predictor = SentimentPredictor()

    print("Починаємо аналіз текстів...")

    for i, text in enumerate(texts, 1):
        print(f"\nТекст {i}:")
        print(f"Зміст: {text}")

        result = predictor.predict(text)

        if result['status'] == 'success':
            print(f"Настрій: {result['sentiment']}")
            print(f"Впевненість: {result['confidence']:.2%}")
            print(f"Необроблене значення: {result['raw_prediction']:.4f}")
        else:
            print(f"Помилка аналізу: {result['error']}")

# Приклад використання
test_texts = [
    "This movie was absolutely fantastic! Great acting and amazing plot.",
    "I really didn't like this film. Poor acting and boring story.",
    "The movie was okay, nothing special but not terrible either."
]

# Створюємо один екземпляр предиктора для всіх текстів
predictor = SentimentPredictor()
analyze_texts(test_texts, predictor)

Починаємо аналіз текстів...

Текст 1:
Зміст: This movie was absolutely fantastic! Great acting and amazing plot.
Настрій: Негативний
Впевненість: 76.94%
Необроблене значення: 0.2306

Текст 2:
Зміст: I really didn't like this film. Poor acting and boring story.
Настрій: Негативний
Впевненість: 74.39%
Необроблене значення: 0.2561

Текст 3:
Зміст: The movie was okay, nothing special but not terrible either.
Настрій: Негативний
Впевненість: 76.00%
Необроблене значення: 0.2400


In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
import json
import os
import shutil
import numpy as np

def save_model_for_tfjs(original_model, save_dir='tfjs_sentiment_model'):
    """
    Saves the model for TensorFlow.js with proper handling of all layer types.

    Args:
        original_model: The trained model to save
        save_dir: Directory where to save the model files
    """
    try:
        print("Starting enhanced model saving process...")

        # Prepare directory
        if os.path.exists(save_dir):
            shutil.rmtree(save_dir)
        os.makedirs(save_dir)

        new_model = original_model

        # Save for TensorFlow.js
        print("\nSaving for TensorFlow.js...")
        tfjs.converters.save_keras_model(new_model, save_dir)

        # Save configuration
        config = {
            'max_len': 200,
            'vocab_size': original_model.get_layer('embedding').input_dim,
            'embedding_dim': original_model.get_layer('embedding').output_dim,
            'model_info': {
                'architecture': 'sequential',
                'lstm1_units': 64,
                'lstm2_units': 32,
                'dense_units': 64
            },
            'word_index': {
                word: idx
                for word, idx in tf.keras.datasets.imdb.get_word_index().items()
                if idx < original_model.get_layer('embedding').input_dim
            }
        }

        config_path = os.path.join(save_dir, 'model_config.json')
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config, f, ensure_ascii=False, indent=2)

        print("\nModel saving process completed successfully!")
        return True

    except Exception as e:
        print(f"\nError during model saving: {str(e)}")
        print("\nDetailed error information:")
        print(f"- TensorFlow version: {tf.__version__}")
        print(f"- TensorFlow.js version: {tfjs.__version__}")
        return False

# Execute the saving process
if 'model' in globals():
    success = save_model_for_tfjs(model)
    if success:
        print("\nNext steps:")
        print("1. Copy all files from tfjs_sentiment_model to your web server")
        print("2. Make sure all files are accessible from your web application")
        print("3. Test the model using the provided web interface")

Starting enhanced model saving process...

Saving for TensorFlow.js...

Model saving process completed successfully!

Next steps:
1. Copy all files from tfjs_sentiment_model to your web server
2. Make sure all files are accessible from your web application
3. Test the model using the provided web interface


Репозиторій - https://github.com/volverina/imdb_reviews

Демонстрація - https://volverina.github.io/imdb_reviews/